In [188]:
import sklearn
import pandas as pd

In [189]:
train = pd.read_excel("train.xlsx")
test = pd.read_excel("test.xlsx")

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        4548 non-null   int64  
 1   предмет           4548 non-null   object 
 2   ФИО               4548 non-null   object 
 3   tutor_head_tags   4548 non-null   object 
 4   tutor_rating      3959 non-null   float64
 5   tutor_reviews     4548 non-null   int64  
 6   description       4547 non-null   object 
 7   experience_desc   2171 non-null   object 
 8   Education_1       4548 non-null   object 
 9   Desc_Education_1  4472 non-null   object 
 10  Education_2       781 non-null    object 
 11  Desc_Education_2  777 non-null    object 
 12  Education_3       106 non-null    object 
 13  Desc_Education_3  105 non-null    object 
 14  Education_4       16 non-null     object 
 15  Desc_Education_4  16 non-null     object 
 16  Ученая степень 1  76 non-null     object 


In [190]:
zero_expel = ["Unnamed: 0"]  # Expel due to non-use
first_expel = ["ФИО", "description"]  # Expel due to low correlation
second_expel = ["experience_desc"]  # Expel due to cannot normalize
third_expel = ["Ученая степень 1", "Ученое звание 1", "Ученая степень 2", "Ученое звание 2"]  # Expel due to little count of examples

train = train.drop(columns=zero_expel + first_expel + second_expel + third_expel)
test = test.drop(columns=zero_expel + first_expel + second_expel + third_expel)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   предмет           4548 non-null   object 
 1   tutor_head_tags   4548 non-null   object 
 2   tutor_rating      3959 non-null   float64
 3   tutor_reviews     4548 non-null   int64  
 4   Education_1       4548 non-null   object 
 5   Desc_Education_1  4472 non-null   object 
 6   Education_2       781 non-null    object 
 7   Desc_Education_2  777 non-null    object 
 8   Education_3       106 non-null    object 
 9   Desc_Education_3  105 non-null    object 
 10  Education_4       16 non-null     object 
 11  Desc_Education_4  16 non-null     object 
 12  status            4535 non-null   object 
 13  experience        4523 non-null   object 
 14  Desc_Education_5  3 non-null      object 
 15  Desc_Education_6  1 non-null      object 
 16  Education_5       3 non-null      object 


In [191]:
# Professionality normalize
education_cols = [col for col in train.columns if col.startswith('Education_')]

train['education_count'] = train[education_cols].notna().sum(axis=1)
test['education_count'] = test[education_cols].notna().sum(axis=1)

education_expel = [f"Education_{i}" for i in range(1, 7)]
education_expel += [f"Desc_Education_{i}" for i in range(1, 7)]

train = train.drop(columns=education_expel)
test = test.drop(columns=education_expel)

# Numeric refreshion
train['experience'] = train['experience'].str.extract('(\\d+)')
test['experience'] = test['experience'].str.extract('(\\d+)')
train['experience'] = pd.to_numeric(train['experience'])
test['experience'] = pd.to_numeric(test['experience'])

train

,предмет,tutor_head_tags,tutor_rating,tutor_reviews,status,experience,categories,mean_price,education_count
0,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",5.0,2,Университетский преподаватель,31.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",16.666667,1
1,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",4.0,7,Школьный преподаватель,8.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",11.666667,2
2,informatika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Школьный преподаватель,11.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",23.333333,3
3,matematika,"['Школьная программа', 'Математический анализ'...",4.0,19,Частный преподаватель,5.0,"['Школьники с 1 по 4 класс', 'Школьники с 5 по...",0.116667,1
4,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Частный преподаватель,19.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",16.666667,1
...,...,...,...,...,...,...,...,...,...
4543,matematika,"['Школьная программа', 'Математический анализ'...",4.0,11,Частный преподаватель,5.0,"['Школьники с 1 по 4 класс', 'Школьники с 5 по...",16.666667,1
4544,informatika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Школьный преподаватель,30.0,['Школьники с 5 по 9 класс'],33.333333,1
4545,informatika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Студент,6.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",23.333333,1
4546,informatika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Студент,1.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",9.722222,1


In [192]:
# Numeric refreshion of vectors
train = train.join(pd.get_dummies(train["предмет"]))
test = test.join(pd.get_dummies(train["предмет"]))
train = train.drop(columns=["предмет"])
test = test.drop(columns=["предмет"])

train = train.join(train['status'].str.get_dummies(sep=','))
test = test.join(test['status'].str.get_dummies(sep=','))
train = train.drop(columns=["status"])
test = test.drop(columns=["status"])

train

,tutor_head_tags,tutor_rating,tutor_reviews,experience,categories,mean_price,education_count,informatika,matematika,Аспирант или ординатор очной формы обучения,Студент,Университетский преподаватель,Частный преподаватель,Школьный преподаватель
0,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",5.0,2,31.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",16.666667,1,False,True,0,0,1,0,0
1,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",4.0,7,8.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",11.666667,2,False,True,0,0,0,0,1
2,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,11.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",23.333333,3,True,False,0,0,0,0,1
3,"['Школьная программа', 'Математический анализ'...",4.0,19,5.0,"['Школьники с 1 по 4 класс', 'Школьники с 5 по...",0.116667,1,False,True,0,0,0,1,0
4,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,19.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",16.666667,1,False,True,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4543,"['Школьная программа', 'Математический анализ'...",4.0,11,5.0,"['Школьники с 1 по 4 класс', 'Школьники с 5 по...",16.666667,1,False,True,0,0,0,1,0
4544,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,30.0,['Школьники с 5 по 9 класс'],33.333333,1,True,False,0,0,0,0,1
4545,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,6.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",23.333333,1,True,False,0,1,0,0,0
4546,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,1.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",9.722222,1,True,False,0,1,0,0,0


In [193]:
# Numeric refreshion of matrix
from sklearn.preprocessing import MultiLabelBinarizer
import ast

train['tutor_head_tags'] = train['tutor_head_tags'].apply(ast.literal_eval)
test['tutor_head_tags'] = test['tutor_head_tags'].apply(ast.literal_eval)

mlb = MultiLabelBinarizer()
tags_dummies = mlb.fit_transform(train['tutor_head_tags'])
tags_dummies_train = pd.DataFrame(tags_dummies, columns=mlb.classes_)
train = pd.concat([train, tags_dummies_train], axis=1)

mlb = MultiLabelBinarizer()
tags_dummies = mlb.fit_transform(test['tutor_head_tags'])
tags_dummies_test = pd.DataFrame(tags_dummies, columns=mlb.classes_)
test = pd.concat([test, tags_dummies_test], axis=1)

train = train.drop('tutor_head_tags', axis=1)
test = test.drop('tutor_head_tags', axis=1)

train['categories'] = train['categories'].apply(ast.literal_eval)
test['categories'] = test['categories'].apply(ast.literal_eval)

mlb = MultiLabelBinarizer()
tags_dummies = mlb.fit_transform(train['categories'])
tags_dummies_train = pd.DataFrame(tags_dummies, columns=mlb.classes_)
train = pd.concat([train, tags_dummies_train], axis=1)

mlb = MultiLabelBinarizer()
tags_dummies = mlb.fit_transform(test['categories'])
tags_dummies_test = pd.DataFrame(tags_dummies, columns=mlb.classes_)
test = pd.concat([test, tags_dummies_test], axis=1)

train = train.drop('categories', axis=1)
test = test.drop('categories', axis=1)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 48 columns):
 #   Column                                                                           Non-Null Count  Dtype  
---  ------                                                                           --------------  -----  
 0   tutor_rating                                                                     3959 non-null   float64
 1   tutor_reviews                                                                    4548 non-null   int64  
 2   experience                                                                       4523 non-null   float64
 3   mean_price                                                                       4548 non-null   float64
 4   education_count                                                                  4548 non-null   int64  
 5   informatika                                                                      4548 non-null   bool   
 6   matemati

In [194]:
train = train.fillna(0)
test = test.fillna(0)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 48 columns):
 #   Column                                                                           Non-Null Count  Dtype  
---  ------                                                                           --------------  -----  
 0   tutor_rating                                                                     4548 non-null   float64
 1   tutor_reviews                                                                    4548 non-null   int64  
 2   experience                                                                       4548 non-null   float64
 3   mean_price                                                                       4548 non-null   float64
 4   education_count                                                                  4548 non-null   int64  
 5   informatika                                                                      4548 non-null   bool   
 6   matemati

In [195]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1516 entries, 0 to 1515
Data columns (total 47 columns):
 #   Column                                                                           Non-Null Count  Dtype  
---  ------                                                                           --------------  -----  
 0   tutor_rating                                                                     1516 non-null   int64  
 1   tutor_reviews                                                                    1516 non-null   int64  
 2   experience                                                                       1516 non-null   float64
 3   education_count                                                                  1516 non-null   int64  
 4   informatika                                                                      1516 non-null   bool   
 5   matematika                                                                       1516 non-null   bool   
 6    Аспиран

In [196]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

# Get union of all columns from both datasets (excluding target)
all_columns = pd.Index(
    train.columns.tolist() + 
    test.columns.tolist()
).unique().drop("mean_price", errors="ignore")

# PROPER DATA SPLITTING
# Training data
y_train = train["mean_price"]
X_train = train.reindex(columns=all_columns, fill_value=0)

# Test data (assuming test doesn't have target)
X_test = test.reindex(columns=all_columns, fill_value=0)

# Initialize and train model
lig_reg = LinearRegression()
lig_reg.fit(X_train, y_train)

# Make predictions
y_train_pred = lig_reg.predict(X_train)  # For training metrics
y_test_pred = lig_reg.predict(X_test)   # For actual test predictions

# Calculate TRAINING metrics
print("Training Metrics:")
print("MAE:", mean_absolute_error(y_train, y_train_pred))
print("MSE:", mean_squared_error(y_train, y_train_pred))
print("R²:", r2_score(y_train, y_train_pred))

# If you had test labels (you don't):
# print("Test Metrics:", mean_absolute_error(y_test_true, y_test_pred))

Training Metrics:
MAE: 6.963244603745574
MSE: 92.24834199992686
R²: 0.16071725501665157


In [197]:
submition = pd.DataFrame(y_test_pred, columns=["mean_price"])
submition = submition.reset_index()

submition.to_csv("my_submit.csv", index=False)